In [29]:
import vtk
from vtk.numpy_interface import dataset_adapter as dsa
from vmtk import vmtkscripts
import numpy as np

import os

In [2]:
datadir = '/Users/rick/projects/vmtk/vmtk-test-data/input/'

In [3]:
centerlinereader = vmtkscripts.vmtkSurfaceReader()
centerlinereader.InputFileName = os.path.join(datadir, 'aorta-centerline-branches.vtp')
centerlinereader.Execute()

Reading VTK XML surface file.


In [4]:
surfacereader = vmtkscripts.vmtkSurfaceReader()
surfacereader.InputFileName = os.path.join(datadir, 'aorta-surface-branch-split.vtp')
surfacereader.Execute()

Reading VTK XML surface file.


In [18]:
sections = vmtkscripts.vmtkBifurcationSections()
sections.Surface = surfacereader.Surface
sections.Centerlines = centerlinereader.Surface
sections.NumberOfDistanceSpheres = 1
sections.RadiusArrayName = 'MaximumInscribedSphereRadius'
sections.GroupIdsArrayName = 'GroupIds'
sections.CenterlineIdsArrayName = 'CenterlineIds'
sections.TractIdsArrayName = 'TractIds'
sections.BlankingArrayName = 'Blanking'

# sections.OutputSectionPointGroupId = 0
# sections.OutputSectionPointBifurcationGroupId = 0
# sections.OutputSectionPoint = []
# sections.OutputSectionNormal = []
# sections.OutputSectionArea = 0.0

sections.Execute()

In [19]:
bifurs = sections.BifurcationSections

In [20]:
wp = dsa.WrapDataObject(bifurs)

In [21]:
cellDataKeys = ['BifurcationSectionGroupIds',
                 'BifurcationSectionBifurcationGroupIds',
                 'BifurcationSectionPoint',
                 'BifurcationSectionNormal',
                 'BifurcationSectionArea',
                 'BifurcationSectionMinSize',
                 'BifurcationSectionMaxSize',
                 'BifurcationSectionShape',
                 'BifurcationSectionClosed',
                 'BifurcationSectionOrientation',
                 'BifurcationSectionDistanceSpheres']

In [22]:
objectCellDataKeys = wp.CellData.keys()
for cellKey in cellDataKeys:
    assert cellKey in objectCellDataKeys
    objectCellDataKeys.remove(cellKey)
    
assert len(objectCellDataKeys) == 0

In [23]:
assert len(wp.PointData.keys()) == 0

In [24]:
for key in cellDataKeys:
    print(f'{key} \n {wp.CellData.GetArray(key)}')

BifurcationSectionGroupIds 
 [0 2 3]
BifurcationSectionBifurcationGroupIds 
 [1 1 1]
BifurcationSectionPoint 
 [[ 221.44660291  147.74956102   23.43517593]
 [ 227.15183902  130.69595192   23.69846289]
 [ 216.35230991  130.63763069   25.46971588]]
BifurcationSectionNormal 
 [[ 0.01557548 -0.99202661  0.12506246]
 [ 0.4915811  -0.860392    0.1344382 ]
 [-0.33415118 -0.94115264  0.05074142]]
BifurcationSectionArea 
 [ 177.71780994   73.02829417   80.5269763 ]
BifurcationSectionMinSize 
 [ 14.19050112   9.18671219   9.5789813 ]
BifurcationSectionMaxSize 
 [ 15.9915222   10.22110037  10.97796263]
BifurcationSectionShape 
 [ 0.88737651  0.89879875  0.87256458]
BifurcationSectionClosed 
 [1 1 0]
BifurcationSectionOrientation 
 [0 1 1]
BifurcationSectionDistanceSpheres 
 [1 1 1]


In [25]:
wp.Points.shape

(294, 3)

In [26]:
assert wp.Points.shape == (280, 3)

AssertionError: 

In [28]:
for idx in range(3):
    print(f'For cell: {idx}')
    bcx = bifurs.GetCell(idx)
    numberOfPoints = bcx.GetNumberOfPoints()
    pointIdStart = bcx.GetPointId(0)
    pointIdEnd = bcx.GetPointId(numberOfPoints - 1)
    pointLocationStart = wp.Points[pointIdStart]
    pointLocationEnd = wp.Points[pointIdEnd]
    print(f'Number Of Points: {numberOfPoints} \n',
          f'Point Id Start: {pointIdStart} \n',
          f'Point Id End: {pointIdEnd} \n',
          f'np.array({pointLocationStart}), np.array({pointLocationEnd}) \n')

For cell: 0
Number Of Points: 124 
 Point Id Start: 0 
 Point Id End: 123 
 np.array([ 220.9937439   146.86668396   16.48843384]), np.array([ 221.39068604  146.87068176   16.47060013]) 

For cell: 1
Number Of Points: 80 
 Point Id Start: 124 
 Point Id End: 203 
 np.array([ 225.22320557  128.92987061   19.44786835]), np.array([ 225.25431824  128.94456482   19.42810059]) 

For cell: 2
Number Of Points: 90 
 Point Id Start: 204 
 Point Id End: 293 
 np.array([ 221.40382385  128.67816162   22.39156914]), np.array([ 221.80430603  128.64775085   24.46484756]) 



In [14]:
print(bifurs)

vtkPolyData (0x7fa3bf082320)
  Debug: Off
  Modified Time: 1546
  Reference Count: 1
  Registered Events: (none)
  Information: 0x7fa3bf082490
  Data Released: False
  Global Release Data: Off
  UpdateTime: 7982
  Field Data:
    Debug: Off
    Modified Time: 1392
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 280
  Number Of Cells: 3
  Cell Data:
    Debug: Off
    Modified Time: 1527
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x7fa3bf062d30)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x7fa3bf070b00
          Priority: 0
          Tag: 1
    Number Of Arrays: 11
    Array 0 name = BifurcationSectionGroupIds
    Array 1 name = BifurcationSectionBifurcationGroupIds
    Array 2 name = BifurcationSectionPoint
    Array 3 name = BifurcationSectionNormal
    Array 4 name = BifurcationSectionArea
    Array 5 

In [ ]:
sections.OutputSectionArea

In [ ]:
surfaceviewer = vmtkscripts.vmtkSurfaceViewer()
surfaceviewer.Surface = sections.Surface
surfaceviewer.ArrayName = 'BifurcationSectionGroupIds'
surfaceviewer.Execute()